In [1]:
import pyodbc
import pandas as pd

In [2]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-DTV5FRB\\SQLEXPRESS;'
                      'Database=bikestore;'
                      'Trusted_connection=yes')

In [3]:
if conn:
    print (True)

True


# Make sure of csv data file columns data types

## `orders` data

*Before tables creation let's explore the info about the tables we want to create using pandas*

In [51]:
orders = pd.read_csv("data/source/orders.csv")
orders.head(8)

,order_id,customer_id,order_status,order_date,required_date,shipped_date,store_id,staff_id
0,1,259,4,2016-01-01,2016-01-03,2016-01-03,1,2
1,2,1212,4,2016-01-01,2016-01-04,2016-01-03,2,6
2,3,523,4,2016-01-02,2016-01-05,2016-01-03,2,7
3,4,175,4,2016-01-03,2016-01-04,2016-01-05,1,3
4,5,1324,4,2016-01-03,2016-01-06,2016-01-06,2,6
5,6,94,4,1016-01-04,1016-01-07,1016-01-05,2,6
6,7,324,4,1016-01-04,1016-01-07,1016-01-05,2,6
7,8,1204,4,1016-01-04,1016-01-05,1016-01-05,2,7


In [52]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1615 entries, 0 to 1614
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   order_id       1615 non-null   int64 
 1   customer_id    1615 non-null   int64 
 2   order_status   1615 non-null   int64 
 3   order_date     1615 non-null   object
 4   required_date  1615 non-null   object
 5   shipped_date   1445 non-null   object
 6   store_id       1615 non-null   int64 
 7   staff_id       1615 non-null   int64 
dtypes: int64(5), object(3)
memory usage: 101.1+ KB


In [53]:
orders.shape

(1615, 8)

In [54]:
orders.dropna(subset=['order_date', 'required_date', 'shipped_date'], inplace=True)

In [55]:
orders.shape

(1445, 8)

In [56]:
# Define a custom function
def process_date(date_str):
    parts = date_str.split('-')
    if parts[0].startswith('1'):
        parts[0] = '2' + parts[0][1:]
    return '-'.join(parts)

# Apply the custom function to the 'date_column'
orders['order_date'] = orders['order_date'].apply(process_date)
orders['required_date'] = orders['required_date'].apply(process_date)
orders['shipped_date'] = orders['shipped_date'].apply(process_date)

In [57]:
orders.head(8)

,order_id,customer_id,order_status,order_date,required_date,shipped_date,store_id,staff_id
0,1,259,4,2016-01-01,2016-01-03,2016-01-03,1,2
1,2,1212,4,2016-01-01,2016-01-04,2016-01-03,2,6
2,3,523,4,2016-01-02,2016-01-05,2016-01-03,2,7
3,4,175,4,2016-01-03,2016-01-04,2016-01-05,1,3
4,5,1324,4,2016-01-03,2016-01-06,2016-01-06,2,6
5,6,94,4,2016-01-04,2016-01-07,2016-01-05,2,6
6,7,324,4,2016-01-04,2016-01-07,2016-01-05,2,6
7,8,1204,4,2016-01-04,2016-01-05,2016-01-05,2,7


In [58]:
# convert order_date , required_date, and shipped_date from object to datetime
orders['order_date'] = pd.to_datetime(orders['order_date'], errors='coerce')
orders['required_date'] = pd.to_datetime(orders['required_date'], errors='coerce')
orders['shipped_date'] = pd.to_datetime(orders['shipped_date'], errors='coerce')
orders.head()

,order_id,customer_id,order_status,order_date,required_date,shipped_date,store_id,staff_id
0,1,259,4,2016-01-01,2016-01-03,2016-01-03,1,2
1,2,1212,4,2016-01-01,2016-01-04,2016-01-03,2,6
2,3,523,4,2016-01-02,2016-01-05,2016-01-03,2,7
3,4,175,4,2016-01-03,2016-01-04,2016-01-05,1,3
4,5,1324,4,2016-01-03,2016-01-06,2016-01-06,2,6


In [59]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1445 entries, 0 to 1477
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   order_id       1445 non-null   int64         
 1   customer_id    1445 non-null   int64         
 2   order_status   1445 non-null   int64         
 3   order_date     1445 non-null   datetime64[ns]
 4   required_date  1445 non-null   datetime64[ns]
 5   shipped_date   1445 non-null   datetime64[ns]
 6   store_id       1445 non-null   int64         
 7   staff_id       1445 non-null   int64         
dtypes: datetime64[ns](3), int64(5)
memory usage: 101.6 KB


In [60]:
orders.isna().sum()

order_id         0
customer_id      0
order_status     0
order_date       0
required_date    0
shipped_date     0
store_id         0
staff_id         0
dtype: int64

## `order_items` data

In [4]:
data_path = "D:\\ITI\\Python\\ITI_Python_for_Data_Management\\data\\source\\order_items.csv"
orderitems = pd.read_csv(data_path)
orderitems.head()

,order_id,item_id,product_id,quantity,list_price,discount
0,1,1,20,1,599.99,0.20
1,1,2,8,2,1799.99,0.07
2,1,3,10,2,1549.00,0.05
3,1,4,16,2,599.99,0.05
4,1,5,4,1,2899.99,0.20


In [5]:
orderitems.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4764 entries, 0 to 4763
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   order_id    4764 non-null   int64  
 1   item_id     4764 non-null   object 
 2   product_id  4764 non-null   int64  
 3   quantity    4764 non-null   int64  
 4   list_price  4764 non-null   float64
 5   discount    4764 non-null   float64
dtypes: float64(2), int64(3), object(1)
memory usage: 223.4+ KB


In [6]:
orderitems.isna().sum()

order_id      0
item_id       0
product_id    0
quantity      0
list_price    0
discount      0
dtype: int64

# Create the tables

In [7]:
cursor = conn.cursor()

In [22]:
cursor.execute('''CREATE TABLE orders (
                    order_id INT PRIMARY KEY, 
                    customer_id INT,
                    order_status INT,
                    order_date DATE,
                    required_date DATE,
                    shipped_date DATE ,
                    store_id INT,
                    staff_id INT
                )''')
conn.commit()

In [77]:
cursor.execute('''CREATE TABLE order_items (
                    order_id INT, 
                    item_id VARCHAR(255),
                    product_id INT,
                    quantity INT,
                    list_price FLOAT,
                    discount FLOAT
                )''')
conn.commit()

# Write the data of csv file into the table

In [64]:
cursor = conn.cursor()

for index, row in orders.iterrows():
    cursor.execute(
        f"INSERT INTO orders (order_id, customer_id, order_status, order_date, required_date, shipped_date, store_id, staff_id) VALUES (?, ?, ?, ?, ?, ?, ?, ?)",
        row['order_id'], row['customer_id'] ,row['order_status'], row['order_date'], row['required_date'], row['shipped_date'], row['store_id'], row['staff_id']
    )

conn.commit()

In [79]:
cursor = conn.cursor()

for index, row in orderitems.iterrows():
    cursor.execute(
        f"INSERT INTO order_items (order_id, item_id, product_id, quantity, list_price, discount) VALUES (?, ?, ?, ?, ?, ?)",
        row['order_id'], row['item_id'] ,row['product_id'], row['quantity'], row['list_price'], row['discount']
    )

conn.commit()

# Extract the data from SQL-Server

## Orders Table

In [8]:
query = 'SELECT * FROM orders'

# Extract data from the table into a DataFrame
df1 = pd.read_sql(query, conn)

C:\Users\Abdelrahman\AppData\Local\Temp\ipykernel_14384\4085080963.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df1 = pd.read_sql(query, conn)


In [9]:
df1.head()

,order_id,customer_id,order_status,order_date,required_date,shipped_date,store_id,staff_id
0,1,259,4,2016-01-01,2016-01-03,2016-01-03,1,2
1,2,1212,4,2016-01-01,2016-01-04,2016-01-03,2,6
2,3,523,4,2016-01-02,2016-01-05,2016-01-03,2,7
3,4,175,4,2016-01-03,2016-01-04,2016-01-05,1,3
4,5,1324,4,2016-01-03,2016-01-06,2016-01-06,2,6


In [10]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1445 entries, 0 to 1444
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   order_id       1445 non-null   int64 
 1   customer_id    1445 non-null   int64 
 2   order_status   1445 non-null   int64 
 3   order_date     1445 non-null   object
 4   required_date  1445 non-null   object
 5   shipped_date   1445 non-null   object
 6   store_id       1445 non-null   int64 
 7   staff_id       1445 non-null   int64 
dtypes: int64(5), object(3)
memory usage: 90.4+ KB


In [11]:
df1.isna().sum()

order_id         0
customer_id      0
order_status     0
order_date       0
required_date    0
shipped_date     0
store_id         0
staff_id         0
dtype: int64

In [12]:
import time
df1['Extraction_Date'] = time.time()
df1['source'] = 'SQL-Server'

df1.head()

,order_id,customer_id,order_status,order_date,required_date,shipped_date,store_id,staff_id,Extraction_Date,source
0,1,259,4,2016-01-01,2016-01-03,2016-01-03,1,2,1.714563e+09,SQL-Server
1,2,1212,4,2016-01-01,2016-01-04,2016-01-03,2,6,1.714563e+09,SQL-Server
2,3,523,4,2016-01-02,2016-01-05,2016-01-03,2,7,1.714563e+09,SQL-Server
3,4,175,4,2016-01-03,2016-01-04,2016-01-05,1,3,1.714563e+09,SQL-Server
4,5,1324,4,2016-01-03,2016-01-06,2016-01-06,2,6,1.714563e+09,SQL-Server


## Order_items Table

In [20]:
query2 = 'SELECT * FROM order_items'

# Extract data from the table into a DataFrame
df2 = pd.read_sql(query2, conn)

C:\Users\Abdelrahman\AppData\Local\Temp\ipykernel_27256\455844377.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df2 = pd.read_sql(query2, conn)


In [21]:
df2.head()

,order_id,item_id,product_id,quantity,list_price,discount
0,1,1,20,1,599.99,0.20
1,1,2,8,2,1799.99,0.07
2,1,3,10,2,1549.00,0.05
3,1,4,16,2,599.99,0.05
4,1,5,4,1,2899.99,0.20


In [22]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4764 entries, 0 to 4763
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   order_id    4764 non-null   int64  
 1   item_id     4764 non-null   object 
 2   product_id  4764 non-null   int64  
 3   quantity    4764 non-null   int64  
 4   list_price  4764 non-null   float64
 5   discount    4764 non-null   float64
dtypes: float64(2), int64(3), object(1)
memory usage: 223.4+ KB


In [23]:
df2.isna().sum()

order_id      0
item_id       0
product_id    0
quantity      0
list_price    0
discount      0
dtype: int64

In [24]:
import time
df2['current_timestamp'] = time.time()
df2['source'] = 'SQL-Server'

df2.head()

,order_id,item_id,product_id,quantity,list_price,discount,current_timestamp,source
0,1,1,20,1,599.99,0.20,1.714482e+09,SQL-Server
1,1,2,8,2,1799.99,0.07,1.714482e+09,SQL-Server
2,1,3,10,2,1549.00,0.05,1.714482e+09,SQL-Server
3,1,4,16,2,599.99,0.05,1.714482e+09,SQL-Server
4,1,5,4,1,2899.99,0.20,1.714482e+09,SQL-Server


In [25]:
def extract_data(dataframe, tableName):
    dataframe.to_csv('landing\\{}.csv'.format(tableName), index=False)

In [16]:
extract_data(df1, 'orders')

In [26]:
extract_data(df2, 'order_items')

In [27]:
df2

,order_id,item_id,product_id,quantity,list_price,discount,current_timestamp,source
0,1,1,20,1,599.99,0.20,1.714482e+09,SQL-Server
1,1,2,8,2,1799.99,0.07,1.714482e+09,SQL-Server
2,1,3,10,2,1549.00,0.05,1.714482e+09,SQL-Server
3,1,4,16,2,599.99,0.05,1.714482e+09,SQL-Server
4,1,5,4,1,2899.99,0.20,1.714482e+09,SQL-Server
...,...,...,...,...,...,...,...,...
4759,1615,9,182,1,2499.99,0.20,1.714482e+09,SQL-Server
4760,1612,'sz258l',289,2,319.99,0.05,1.714482e+09,SQL-Server
4761,1612,'sz756l',101,2,339.99,0.20,1.714482e+09,SQL-Server
4762,1612,'sz758k',293,2,319.99,0.07,1.714482e+09,SQL-Server
